In [18]:
import os
import pandas as pd
from langchain_openai import AzureOpenAIEmbeddings
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAI

from dotenv import load_dotenv
load_dotenv(verbose=True, override=True)

embeddings_api = AzureOpenAIEmbeddings(azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"),
                                    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION")
                                    )

llm_chat_api = AzureChatOpenAI(openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
                            azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
                            max_tokens = 3000
                            )

llm_instruct_api = AzureOpenAI(openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
                            azure_deployment=os.environ["AZURE_OPENAI_INSTRUCT_DEPLOYMENT_NAME"],
                            max_tokens = 1500
                            )

card_name = "Capital One"


In [19]:
import os
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate
    )
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

model = llm_chat_api

# ---- DEFINE OUTPUT DATA TYPES WITHIN THE CLASS AND INITILIAZE A PARSER ----
class Rewards(BaseModel):

    values: list = Field(description='Python list of dictionaries containing CardName, Discount and Description')

parser = PydanticOutputParser(pydantic_object=Rewards)

# ---------------------------- SETUP THE REQUEST -----------------------------
human_prompt = HumanMessagePromptTemplate.from_template("{request}\n{format_instructions}")
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

request = chat_prompt.format_prompt(
    request=''' Give me 10 ''' + card_name + ''' credit card memberships promotions for members, find as much real one as you can and the rest feel free to generate similiar to the examples. 
    Ouput in the following columns schema: card name, discount, description
    example: [{ CardName: American Express Gold Card, Discount: €30, Description:Get a €30 statement credit by spending €150 or more at Ray-Ban. Offer valid until 9/6/2024.},
    { CardName: American Express Gold Card, Discount: €25, Description:Earn a €25 statement credit with a €99 purchase on an annual subscription at MAX. Offer expires on 9/30/2024.},
    { CardName: American Express Cash Magnet Card, Discount: 1.5 cash back, Description:Earn unlimited 1.5% cash back on all purchases.}]
    ''',
    format_instructions=parser.get_format_instructions()
).to_messages()

results = model(request, temperature=0)
results_values = parser.parse(results.content)  # Player class object


# ----------------------------- SHOW THE RESULTS -----------------------------
import pandas as pd

results_dataframe = pd.DataFrame.from_dict(results_values.values)

results_dataframe

,CardName,Discount,Description
0,Capital One Venture Rewards Credit Card,"60,000 bonus miles","Earn 60,000 bonus miles after spending $3,000 ..."
1,Capital One Quicksilver Cash Rewards Credit Card,$200 cash bonus,Earn a one-time $200 cash bonus after you spen...
2,Capital One SavorOne Cash Rewards Credit Card,3% cash back,"Earn unlimited 3% cash back on dining, enterta..."
3,Capital One Platinum Credit Card,No annual fee,Enjoy no annual fee and access to a higher cre...
4,Capital One VentureOne Rewards Credit Card,"20,000 bonus miles","Earn 20,000 bonus miles once you spend $500 on..."
5,Capital One Walmart Rewards Card,5% cash back,"Earn 5% cash back on Walmart.com purchases, in..."
6,Capital One Spark Cash for Business,$500 cash bonus,"Earn a $500 cash bonus when you spend $4,500 i..."
7,Capital One Spark Miles for Business,"50,000 bonus miles","Earn a one-time bonus of 50,000 miles – equal ..."
8,Capital One QuicksilverOne Cash Rewards Credit...,1.5% cash back,Earn unlimited 1.5% cash back on every purchas...
9,Capital One Secured Mastercard,Credit building,Build your credit with responsible use and aut...


In [20]:
results_dataframe.to_pickle(os.path.join(os.getcwd(), "data/rewards", card_name + ".pkl"))